In [4]:
!pip install gradio pandas matplotlib openpyxl -q

import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime

DATA_FILE = "bmi_data.csv"
USERS_FILE = "users.csv"

COLUMNS = [
    "username",
    "age",
    "gender",
    "height_cm",
    "weight_kg",
    "bmi",
    "category",
    "datetime"
]

def load_data():
    if os.path.exists(DATA_FILE):
        try:
            df = pd.read_csv(DATA_FILE)
            for c in COLUMNS:
                if c not in df.columns:
                    df[c] = None
            return df[COLUMNS]
        except:
            return pd.DataFrame(columns=COLUMNS)
    return pd.DataFrame(columns=COLUMNS)

def save_data(df):
    df.to_csv(DATA_FILE, index=False)

def load_users():
    if os.path.exists(USERS_FILE):
        try:
            df = pd.read_csv(USERS_FILE)
            if "username" not in df.columns:
                return pd.DataFrame(columns=["username","password"])
            return df
        except:
            return pd.DataFrame(columns=["username","password"])
    return pd.DataFrame(columns=["username","password"])

def save_users(df):
    df.to_csv(USERS_FILE, index=False)

# --------------------------
# LOGIN / REGISTER
# --------------------------
def login_or_register(username, password):
    username = (username or "").strip()
    if not username:
        return "Please enter a username.", "", ""
    if not password:
        return "Please enter a password.", "", ""

    df = load_users()
    mask = df["username"].astype(str).str.lower() == username.lower()

    if mask.any():
        row = df[mask].iloc[0]
        if str(row["password"]) == str(password):
            return f"Logged in as {row['username']}", row["username"], row["username"]
        return "Incorrect password.", "", ""

    new_row = {"username": username, "password": password}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    save_users(df)
    return f"User '{username}' registered successfully.", username, username

# --------------------------
# BMI CALCULATIONS
# --------------------------
def calculate_bmi(weight_kg, height_cm):
    height_m = height_cm / 100
    return round(weight_kg / (height_m ** 2), 2)

def classify_bmi(bmi):
    if bmi < 18.5: return "Underweight"
    if bmi < 25: return "Normal weight"
    if bmi < 30: return "Overweight"
    return "Obese"

def advice_for_category(category):
    if category == "Underweight":
        return "Increase calorie intake with healthy foods."
    if category == "Normal weight":
        return "Great! Maintain your lifestyle."
    if category == "Overweight":
        return "Try regular exercise and healthy diet."
    return "Consult a healthcare provider for guidance."

def calculate_and_save(username, age, gender, height_cm, weight_kg):
    if not username:
        return "Login first.", "", "", pd.DataFrame(columns=COLUMNS)

    df = load_data()
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    bmi = calculate_bmi(weight_kg, height_cm)
    category = classify_bmi(bmi)

    row = {
        "username": username,
        "age": age,
        "gender": gender,
        "height_cm": height_cm,
        "weight_kg": weight_kg,
        "bmi": bmi,
        "category": category,
        "datetime": now
    }

    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    save_data(df)

    return (
        "Saved successfully.",
        f"{username}, your BMI is {bmi} ({category}).",
        advice_for_category(category),
        pd.DataFrame([row])
    )

# --------------------------
# HISTORY + TREND PLOT
# --------------------------
def get_user_history(username):
    if not username:
        return "Login first.", pd.DataFrame(columns=COLUMNS), None

    df = load_data()
    user_df = df[df["username"] == username]

    if user_df.empty:
        return "No history found.", pd.DataFrame(columns=COLUMNS), None

    user_df["datetime_parsed"] = pd.to_datetime(user_df["datetime"])
    user_df = user_df.sort_values("datetime_parsed")

    fig = plt.figure()
    colors = {
        "Underweight": "tab:blue",
        "Normal weight": "tab:green",
        "Overweight": "tab:orange",
        "Obese": "tab:red"
    }

    for cat in user_df["category"].unique():
        part = user_df[user_df["category"] == cat]
        plt.plot(part["datetime_parsed"], part["bmi"], marker="o",
                 label=cat, color=colors.get(cat, None))

    plt.title(f"BMI Trend for {username}")
    plt.xticks(rotation=45)
    plt.ylabel("BMI")
    plt.xlabel("Date")
    plt.legend(title="BMI Category")
    plt.tight_layout()

    return f"Loaded {username}'s history.", user_df[COLUMNS], fig

# --------------------------
# CLEAR HISTORY FOR USER
# --------------------------
def clear_my_history(username):
    if not username:
        return "Login first.", pd.DataFrame(columns=COLUMNS)

    df = load_data()
    df = df[df["username"] != username]
    save_data(df)

    return f"{username}'s history cleared.", df

# --------------------------
# CLEAR ALL DATA
# --------------------------
def clear_all_data():
    save_data(pd.DataFrame(columns=COLUMNS))
    return "All BMI data cleared.", pd.DataFrame(columns=COLUMNS)

# --------------------------
# EXPORT TO EXCEL
# --------------------------
def get_all_data():
    df = load_data()
    return f"Total {len(df)} records.", df

def export_excel():
    df = load_data()
    file = "bmi_export.xlsx"
    df.to_excel(file, index=False)
    return file

# ============================
#        UI STARTS HERE
# ============================

with gr.Blocks(title="BMI System") as demo:

    gr.Markdown("# 🧮 BMI Calculator with Login, Trends, Export & Clear Buttons")

    current_user = gr.State("")

    # LOGIN TAB
    with gr.Tab("Login"):
        u = gr.Textbox(label="Username")
        p = gr.Textbox(label="Password", type="password")
        btn = gr.Button("Login / Register", variant="primary")
        status = gr.Textbox(label="Status", interactive=False)
        display_user = gr.Textbox(label="Logged-in User", interactive=False)

        btn.click(
            login_or_register, [u, p],
            [status, current_user, display_user]
        )

    # BMI TAB
    with gr.Tab("BMI Calculator"):
        show_user = gr.Textbox(label="User", interactive=False)
        age = gr.Number(label="Age", value=25)
        gender = gr.Radio(["Male", "Female", "Other"], label="Gender")
        height = gr.Number(label="Height (cm)", value=170)
        weight = gr.Number(label="Weight (kg)", value=70)
        calc_btn = gr.Button("Calculate & Save", variant="primary")

        out1 = gr.Textbox(label="Status")
        out2 = gr.Textbox(label="BMI Result")
        out3 = gr.Textbox(label="Advice")
        out4 = gr.Dataframe(label="Latest")

        demo.load(lambda x: x, current_user, show_user)
        btn.click(lambda x: x, current_user, show_user)

        calc_btn.click(
            calculate_and_save,
            [current_user, age, gender, height, weight],
            [out1, out2, out3, out4]
        )

    # HISTORY TAB
    with gr.Tab("My History"):
        hist_status = gr.Textbox(label="Status", interactive=False)
        hist_table = gr.Dataframe(label="My Data")
        hist_plot = gr.Plot(label="My Trend")
        load_hist_btn = gr.Button("Load My History")

        clear_my_btn = gr.Button("🗑 Clear My History", variant="stop")

        load_hist_btn.click(
            get_user_history, [current_user],
            [hist_status, hist_table, hist_plot]
        )

        clear_my_btn.click(
            clear_my_history, [current_user],
            [hist_status, hist_table]
        )

    # ALL DATA TAB
    with gr.Tab("All Data & Export"):
        all_status = gr.Textbox(label="Status")
        all_table = gr.Dataframe(label="All Records")
        refresh_btn = gr.Button("Refresh Data")

        export_btn = gr.Button("Export to Excel")
        excel_file = gr.File()

        clear_all_btn = gr.Button("🗑 Clear ALL Data", variant="stop")

        refresh_btn.click(get_all_data, [], [all_status, all_table])
        export_btn.click(export_excel, [], excel_file)

        clear_all_btn.click(
            clear_all_data, [], [all_status, all_table]
        )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bcfaa49b6b37a4751d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
